In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pubg-finish-placement-prediction/train_V2.csv
/kaggle/input/pubg-finish-placement-prediction/test_V2.csv
/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv


In [2]:
import numpy as np 
import pandas as pd 
import warnings 
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

train = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')
test = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')

In [3]:
train.drop(['Id', 'groupId', 'matchId', 'matchType'], axis = 1, inplace = True)
test.drop(['Id', 'groupId', 'matchId', 'matchType'], axis = 1, inplace = True)

In [4]:
X = train.drop('winPlacePerc', axis = 1) 
y = train['winPlacePerc']

from sklearn.model_selection import train_test_split 
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size = 0.2)

In [5]:
import lightgbm as lgb
from sklearn.model_selection import cross_validate
lgbm_reg = lgb.LGBMRegressor(random_state = 42, 
                             metric = 'mae')
lgbm_reg.fit(X_train, y_train, 
             eval_set = [(X_train, y_train), 
                         (X_valid, y_valid)], 
             verbose = 10) 
pred = lgbm_reg.predict(test)

[10]	training's l1: 0.122748	valid_1's l1: 0.122769
[20]	training's l1: 0.0815657	valid_1's l1: 0.0815957
[30]	training's l1: 0.070105	valid_1's l1: 0.0701726
[40]	training's l1: 0.0662294	valid_1's l1: 0.066322
[50]	training's l1: 0.0638306	valid_1's l1: 0.0639308
[60]	training's l1: 0.0625945	valid_1's l1: 0.0626938
[70]	training's l1: 0.0617079	valid_1's l1: 0.0618071
[80]	training's l1: 0.0610996	valid_1's l1: 0.0612045
[90]	training's l1: 0.060659	valid_1's l1: 0.0607638
[100]	training's l1: 0.0603256	valid_1's l1: 0.0604342


In [6]:
submission = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv')
submission['winPlacePerc'] = pred
submission.to_csv('submission.csv', index = False)